<a href="https://colab.research.google.com/github/SuhedaTaspinar/skin-cancer-classification/blob/main/skinCancerClassificationModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files


from google.colab import drive
drive.mount('/content/drive')

data_dir1 = '/content/drive/My Drive/archive/HAM10000_images_part_1/'
data_dir2 = '/content/drive/My Drive/archive/HAM10000_images_part_2/'
metadata_file = '/content/drive/My Drive/archive/HAM10000_metadata.csv'


metadata = pd.read_csv(metadata_file)

image_paths = []
labels = metadata['dx'].values

for img_id in metadata['image_id']:
    img_path1 = os.path.join(data_dir1, f"{img_id}.jpg")
    img_path2 = os.path.join(data_dir2, f"{img_id}.jpg")

    if os.path.exists(img_path1):
        image_paths.append(img_path1)
    elif os.path.exists(img_path2):
        image_paths.append(img_path2)
    else:
        print(f"Görüntü yolu bulunamadı: {img_id}")

label_dict = {label: idx for idx, label in enumerate(np.unique(labels))}
labels = np.array([label_dict[label] for label in labels])

img_size = 64

images = []
for img_path in image_paths:
    img = cv2.imread(img_path)
    if img is None:
        print(f"Görüntü yüklenemedi: {img_path}")
        continue

    img = cv2.resize(img, (img_size, img_size))
    images.append(img)

images = np.array(images)
labels = to_categorical(labels, num_classes=len(label_dict))

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_dict), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=20,
    steps_per_epoch=len(X_train) // 32
)

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Doğruluğu: {test_accuracy * 100:.2f}%")

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kayıp')
plt.legend()

plt.show()
model_path_h5 = '/content/drive/My Drive/HAM10000_skin_cancer_model.h5'
model_path_tf = '/content/drive/My Drive/HAM10000_skin_cancer_model'

model.save(model_path_h5)
model.save(model_path_tf)
print(f"Model .h5 formatında {model_path_h5} yoluna kaydedildi.")
print(f"Model TensorFlow formatında {model_path_tf} yoluna kaydedildi.")

model_path_keras = '/content/HAM10000_skin_cancer_model.keras'
model.save(model_path_keras)
files.download(model_path_keras)
print("Model yerel bilgisayara indirildi.")